In [ ]:
import os
import re
import glob
import urllib
import subprocess
import pandas as pd
import numpy as np

from io import StringIO

from Bio import SeqIO
from Bio import Entrez
from Bio import AlignIO
from Bio.Align.Applications import MuscleCommandline

In [ ]:
# setup path 
pd.set_option('display.max_rows', 10000)
os.chdir('/Users/rootqz/Desktop/ReyLab/project/BSH/')

In [ ]:
### Build HMM using BSH gene cluster from Foley et al. 2023 Nat. Microbiol. paper

# load bsh gene files
cluster_seq = pd.read_csv('data/Foley23_cluster_seq.csv', sep = ',')
cluster_meta = pd.read_csv('data/Foley23_cluster_meta.csv', sep = ',')
cloned_seq = pd.read_csv('data/Foley23_cloned_bsh.csv', sep = ',')

# write out BSH cluster aa sequence fasta file
with open('data/Foley23_cluster_seq.fasta', 'w') as out:
    for i in range(cluster_seq.shape[0]):
        print('>{}'.format(cluster_seq.cluster_name[i]), file=out)
        print('{}'.format(cluster_seq.aa_seq[i]), file=out)

# mulyiple alignment via MUSCLE
subprocess.call('muscle3.8.31_i86darwin64 -in data/Foley23_cluster_seq.fasta \
                                          -out data/Foley23_cluster_seq.afa', stdout=True, shell=True)

# convert to stockholm format
hmm_bsh = SeqIO.parse('data/Foley23_cluster_seq.afa', 'fasta')
SeqIO.write(hmm_bsh, 'data/Foley23_cluster_seq.sto', 'stockholm')

# build hmm profile
subprocess.call('hmmbuild data/Foley23_cluster_seq.hmm \
                          data/Foley23_cluster_seq.sto', stdout=True, shell=True)

# build blast reference
subprocess.call('makeblastdb -in data/blast_ref/Foley23_cluster_seq.fasta -dbtype prot', stdout=True, shell=True)

In [ ]:
### Download CDS from bacteria strain genome

# load strain list
strain_list = pd.read_csv('data/strain_list.csv', sep = ',', index_col=0, na_values=None)

# ncbi genomes list 
gcf_list = [x.split('/')[-2] for x in strain_list.ncbi_ftp.to_list() if isinstance(x, str)]

# fetch gene sequence
os.chdir('/Users/rootqz/Desktop/ReyLab/project/BSH/data/genome_faa/')

for i in range(strain_list.shape[0]):
    
    # filter out NA value, i.e., no RefSeq genome from NCBI
    if isinstance(strain_list.ncbi_ftp.to_list()[i], str):
        GCF_ID = strain_list.ncbi_ftp.to_list()[i].split('/')[-2]
        #print(GCF_ID)
        
        # download faa from RefSeq
        subprocess.call('wget {}{}_protein.faa.gz'.format(strain_list.ncbi_ftp.to_list()[i], GCF_ID), stdout=True, shell=True)
        # unzip
        subprocess.call('gunzip {}_protein.faa.gz'.format(GCF_ID), stdout=True, shell=True)

In [ ]:
### BLASTp to BSH gene cluster 

for gcf in gcf_list:
    
    query_file = 'data/genome_faa/{}_protein.faa'.format(gcf)
    out_file = 'data/blastp_out/{}.blastp'.format(gcf)
    
    # BLASTp
    subprocess.call('blastp -query {} -db data/blast_ref/Foley23_cluster_seq.fasta -out {} -evalue 1e-5 -outfmt "6 qseqid qlen sseqid slen pident nident ppos positive length mismatch gapopen qstart qend sstart send evalue bitscore"'.format(query_file, out_file), stdout=True, shell=True)

# concat BLASTp results into single df
blastp_merge= pd.DataFrame()

for gcf in gcf_list:
    blastp_out_file = 'data/blastp_out/{}.blastp'.format(gcf)
    if os.stat(blastp_out_file).st_size > 0:
        blastp_out = pd.read_csv(blastp_out_file, sep = '\t', header=None)
        blastp_out.columns = ['qseqid', 'qlen', 'sseqid', 'slen', 
                              'pident', 'nident', 'ppos', 'positive', 'length', 'mismatch', 'gapopen', 
                              'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore']
        
        # add new column to store genome (GCF ID) info
        blastp_out['gcfID'] = gcf
        
        if blastp_merge.shape[0] == 0:
            blastp_merge = blastp_out
        else:
            blastp_merge = pd.concat([blastp_merge, blastp_out], axis=0)

In [ ]:
### HMM
for gcf in gcf_list:
    
    query_file = 'data/genome_faa/{}_protein.faa'.format(gcf)
    out_file = 'data/hmm_out/{}.hmm.out'.format(gcf)
    
    # BLASTp
    subprocess.call('hmmsearch --tblout {} data/Foley23_cluster_seq.hmm {}'.format(out_file, query_file), stdout=True, shell=True)  

# concat HMM results into single df
hmm_merge = pd.DataFrame()

for gcf in gcf_list:
    hmm_out_file = 'data/hmm_out/{}.hmm.out'.format(gcf)
    
    # test if no HMM hit, thus hmm out file contains only comment lines
    dummy_pd = pd.read_csv(hmm_out_file, sep='\t', header=None)
    
    if dummy_pd.shape[0] > 13:
        hmm_out = pd.read_csv(hmm_out_file, delim_whitespace=True, comment='#', header=None, usecols=range(18))
        
        hmm_out.columns = ['tname', 'tacc', 'qname', 'qacc', 
                           'full_evalue', 'full_score', 'full_bias', 
                           'domain_evalue', 'domain_score', 'domain_bias', 
                           'exp', 'reg', 'clu', 'ov', 'env', 'dom', 'rep', 'inc']
        
        # add new column to store genome (GCF ID) info
        hmm_out['gcfID'] = gcf
        
        if hmm_merge.shape[0] == 0:
            hmm_merge = hmm_out
        else:
            hmm_merge = pd.concat([hmm_merge, hmm_out], axis=0)

# get gene by BLASTp criteria: gene_length > 300 and < 400, piden > 30
blast_filter = blastp_merge[(blastp_merge['qlen'] > 300) & (blastp_merge['qlen'] < 400) & (blastp_merge['pident'] > 25)]
blast_filter_nr = list(set(blast_filter['qseqid'].to_list()))

# filter hit
hmm_filter = hmm_merge[(hmm_merge['full_score'] > 100) & (hmm_merge['tname'].isin(blast_filter_nr))]

bsh_hits = []
bsh_seqID = []

for i in range(hmm_filter.shape[0]):
    gcf = hmm_filter['gcfID'].to_list()[i]
    gene = hmm_filter['tname'].to_list()[i]

    if gene not in bsh_seqID:
        faa_file = 'data/genome_faa/{}_protein.faa'.format(gcf)
        faa_seqs = SeqIO.parse(faa_file, 'fasta')
        
        # append all record to a list
        bsh_hits = bsh_hits + [seq for seq in faa_seqs if seq.id == gene]
        bsh_seqID = bsh_seqID+[gene]

SeqIO.write(bsh_hits, 'data/BSH_hmm_hits.fasta', 'fasta')

# merge gene acc with bacteria name
strain_list_merge = strain_list[strain_list['ncbi_ftp'].notna()]
strain_list_merge['gcfID'] = strain_list_merge['ncbi_ftp'].str.split('/', expand=True)[9]


motif_merge = pd.merge(hmm_filter[['tname', 'full_evalue', 'full_score', 'gcfID']], 
                       strain_list_merge[['gcfID', 'name', 'phylum']], 
                       how='inner', 
                       on='gcfID')

strain_list_merge[['gcfID', 'name', 'phylum']]

In [ ]:
### Build HMM using PVA gene from O’Flaherty et al. 2018 mSphere paper

# mulyiple alignment via MUSCLE
subprocess.call('muscle3.8.31_i86darwin64 -in data/PVA_ref.fasta \
                                          -out data/PVA_ref.afa', stdout=True, shell=True)

# convert to stockholm format
hmm_bsh = SeqIO.parse('data/PVA_ref.afa', 'fasta')
SeqIO.write(hmm_bsh, 'data/PVA_ref.sto', 'stockholm')

# build hmm profile
subprocess.call('hmmbuild data/PVA_ref.hmm \
                          data/PVA_ref.sto', stdout=True, shell=True)


# hmm search
subprocess.call('hmmsearch --tblout data/PVA_BSHhit.hmm.out data/PVA_ref.hmm data/BSH_hmm_hits.fasta', stdout=True, shell=True)  

# convert hmm out table
hmm_out = pd.read_csv('data/PVA_BSHhit.hmm.out', delim_whitespace=True, comment='#', header=None, usecols=range(18))

hmm_out.columns = ['tname', 'tacc', 'qname', 'qacc', 
                   'full_evalue', 'full_score', 'full_bias', 
                   'domain_evalue', 'domain_score', 'domain_bias', 
                   'exp', 'reg', 'clu', 'ov', 'env', 'dom', 'rep', 'inc']

hmm_out.to_csv('data/PVA_BSHhit_hmm.tsv', sep = '\t', index=False)

# hmm search of PVA ref seq
subprocess.call('hmmsearch --tblout data/PVA_RefGene.hmm.out data/PVA_ref.hmm data/PVA_ref.fasta', stdout=True, shell=True)  

# convert hmm out table
hmm_out = pd.read_csv('data/PVA_RefGene.hmm.out', delim_whitespace=True, comment='#', header=None, usecols=range(18))

hmm_out.columns = ['tname', 'tacc', 'qname', 'qacc', 
                   'full_evalue', 'full_score', 'full_bias', 
                   'domain_evalue', 'domain_score', 'domain_bias', 
                   'exp', 'reg', 'clu', 'ov', 'env', 'dom', 'rep', 'inc']

hmm_out.to_csv('data/PVA_RefGene_hmm.tsv', sep = '\t', index=False)